# Hierarchical Normal vs Hierarchical Regression (Bike Sharing)

This Colab notebook fits two Bayesian models to the daily bike counts:

1. **Hierarchical Normal model** (by season)
2. **Hierarchical Regression model** (by season with predictors)

It then checks convergence (R-hat, ESS) and compares models using LOO.

In [1]:

!pip install cmdstanpy arviz pandas numpy -q

import cmdstanpy as cp
cp.install_cmdstan()


CmdStan install directory: /root/.cmdstan
Installing CmdStan version: 2.37.0
Download successful, file: /tmp/tmper3_0i75
Extracting distribution
Unpacked download as cmdstan-2.37.0
Building version cmdstan-2.37.0, may take several minutes, depending on your system.
Installed cmdstan-2.37.0
Test model compilation


True

In [2]:
# Upload day.csv (from your project's data/day.csv)
from google.colab import files
import io
import pandas as pd

print('Please upload your day.csv (from data/day.csv in your project).')
uploaded = files.upload()

if 'day.csv' not in uploaded:
    raise ValueError('You must upload a file named \"day.csv\".')

day = pd.read_csv(io.BytesIO(uploaded['day.csv']))
print('Loaded day.csv with shape:', day.shape)
day.head()


Please upload your day.csv (from data/day.csv in your project).


Saving day.csv to day.csv
Loaded day.csv with shape: (731, 16)


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [3]:
# Prepare data for the two models
import numpy as np

required_cols = ['cnt', 'season', 'temp', 'atemp', 'hum', 'windspeed', 'holiday', 'workingday', 'weathersit']
missing = [c for c in required_cols if c not in day.columns]
if missing:
    raise ValueError(f'Missing required columns in day.csv: {missing}')

# Sort by date if available (not essential, but nice)
if 'dteday' in day.columns:
    day['dteday'] = pd.to_datetime(day['dteday'])
    day = day.sort_values('dteday').reset_index(drop=True)

y = day['cnt'].to_numpy(dtype=float)
season = day['season'].to_numpy(dtype=int)

N = y.shape[0]
K = int(season.max())  # seasons 1..K
print(f'N = {N}, K = {K}')

def zscore(x):
    x = np.asarray(x, dtype=float)
    return (x - x.mean()) / (x.std() + 1e-10)

temp_std = zscore(day['temp'])
atemp_std = zscore(day['atemp'])
hum_std = zscore(day['hum'])
windspeed_std = zscore(day['windspeed'])
holiday = day['holiday'].to_numpy(dtype=float)
workingday = day['workingday'].to_numpy(dtype=float)
weathersit = day['weathersit'].to_numpy(dtype=float)

X = np.column_stack([
    temp_std,
    atemp_std,
    hum_std,
    windspeed_std,
    holiday,
    workingday,
    weathersit,
])
P = X.shape[1]
print('Design matrix X shape:', X.shape)
print('First row of X:', X[0])


N = 731, K = 4
Design matrix X shape: (731, 7)
First row of X: [-0.82666213 -0.67994602  1.25017133 -0.38789169  0.          0.
  2.        ]


In [6]:
# Write Stan code for both models into the Colab filesystem
hierarchical_normal_stan = r"""
// ============================================================================
// Hierarchical Normal Model for Daily Bike Counts by Season
// ============================================================================
data {
  int<lower=1> N;                     // number of observations
  int<lower=1> K;                     // number of seasons (groups)
  vector[N] y;                        // daily counts
  array[N] int<lower=1, upper=K> group; // season index for each day
}

parameters {
  real mu;                            // overall mean
  real<lower=0> tau;                  // between-season precision
  vector[K] theta;                    // season means
  real<lower=0> sigma;                // within-season SD
}

model {
  // Hyperpriors (data-informed, moderately tight)
  // Counts are around 4500 with sd ~ 2000
  mu ~ normal(4500, 500);
  tau ~ gamma(2, 0.5);                // mean ~ 4, avoids extreme values
  theta ~ normal(mu, 1.0 / sqrt(tau));
  sigma ~ normal(0, 1500);            // half-normal via <lower=0>

  // Likelihood
  for (n in 1:N) {
    y[n] ~ normal(theta[group[n]], sigma);
  }
}

generated quantities {
  vector[N] log_lik;
  for (n in 1:N) {
    log_lik[n] = normal_lpdf(y[n] | theta[group[n]], sigma);
  }
}
"""


hierarchical_regression_stan = r"""
// ============================================================================
// Hierarchical Regression (Normal likelihood) by Season
// Y_ni ~ Normal(alpha_group[n] + X_n * beta_group[n], sigma)
// ============================================================================
data {
  int<lower=1> N;                     // number of observations
  int<lower=1> K;                     // number of groups (seasons)
  int<lower=1> P;                     // number of predictors
  matrix[N, P] X;                     // standardized predictors
  array[N] int<lower=1, upper=K> group; // group index
  vector[N] y;                        // response (daily counts)
}

parameters {
  // Hierarchical intercepts
  real mu_alpha;                      // overall intercept mean
  real<lower=0> tau_alpha;            // between-group precision
  vector[K] alpha;                    // group intercepts

  // Hierarchical regression coefficients
  vector[P] mu_beta;                  // overall coefficient means
  vector<lower=0>[P] tau_beta;        // coefficient precisions
  matrix[K, P] beta;                  // group-specific coefficients

  // Residual sd
  real<lower=0> sigma;
}

transformed parameters {
  vector[N] mu;
  for (n in 1:N) {
    mu[n] = alpha[group[n]] + X[n] * beta[group[n]]';
  }
}

model {
  // Priors for intercept hierarchy
  mu_alpha ~ normal(4500, 500);
  tau_alpha ~ gamma(2, 0.5);
  alpha ~ normal(mu_alpha, 1.0 / sqrt(tau_alpha));

  // Priors for regression hierarchy
  mu_beta ~ normal(0, 0.5);
  tau_beta ~ gamma(2, 0.5);

  {
    vector[P] sd_beta;
    sd_beta = 1.0 ./ sqrt(tau_beta);
    for (k in 1:K) {
      beta[k] ~ normal(mu_beta, sd_beta);
    }
  }

  // Residual sd
  sigma ~ normal(0, 1500);

  // Likelihood
  y ~ normal(mu, sigma);
}

generated quantities {
  vector[N] log_lik;
  for (n in 1:N) {
    log_lik[n] = normal_lpdf(y[n] | mu[n], sigma);
  }
}
"""


with open('hierarchical_normal.stan', 'w') as f:
    f.write(hierarchical_normal_stan)

with open('hierarchical_regression.stan', 'w') as f:
    f.write(hierarchical_regression_stan)

print('Wrote hierarchical_normal.stan and hierarchical_regression.stan')


Wrote hierarchical_normal.stan and hierarchical_regression.stan


In [7]:
# Fit both models with CmdStanPy
from cmdstanpy import CmdStanModel

stan_data_common = {
    'N': int(N),
    'K': int(K),
    'y': y.astype(float),
    'group': season.astype(int),
}

print('=== Compiling Hierarchical Normal Model ===')
model_normal = CmdStanModel(stan_file='hierarchical_normal.stan')

print('\n=== Sampling Hierarchical Normal Model ===')
fit_normal = model_normal.sample(
    data=stan_data_common,
    chains=4,
    parallel_chains=4,
    iter_warmup=2000,
    iter_sampling=2000,
    seed=123,
    adapt_delta=0.99,
    max_treedepth=15,
)

print('\n=== Compiling Hierarchical Regression Model ===')
model_reg = CmdStanModel(stan_file='hierarchical_regression.stan')

print('\n=== Sampling Hierarchical Regression Model ===')
stan_data_reg = {
    **stan_data_common,
    'P': int(P),
    'X': X.astype(float),
}

fit_reg = model_reg.sample(
    data=stan_data_reg,
    chains=4,
    parallel_chains=4,
    iter_warmup=2000,
    iter_sampling=2000,
    seed=456,
    adapt_delta=0.99,
    max_treedepth=15,
)

print('\nSampling complete for both models.')


=== Compiling Hierarchical Normal Model ===

=== Sampling Hierarchical Normal Model ===


chain 1:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

chain 2:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

chain 3:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

chain 4:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

...

chain 2:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

chain 3:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

chain 4:   0%|          | 0/4000 [00:00<?, ?it/s, (Warmup)]

Exception: gamma_lpdf: Random variable is 0, but must be positive finite! (in 'hierarchical_regression.stan', line 40, column 2 to column 28)
Exception: gamma_lpdf: Random variable[2] is 0, but must be positive finite! (in 'hierarchical_regression.stan', line 45, column 2 to column 27)
Exception: gamma_lpdf: Random variable[5] is 0, but must be positive finite! (in 'hierarchical_regression.stan', line 45, column 2 to column 27)
Exception: gamma_lpdf: Random variable is 0, but must be positive finite! (in 'hierarchical_regression.stan', line 40, column 2 to column 28)
Consider re-running with show_console=True if the above output is unclear!




Sampling complete for both models.


In [8]:
# Convert to ArviZ and check convergence
import arviz as az

print('Converting to ArviZ InferenceData...')
idata_normal = az.from_cmdstanpy(posterior=fit_normal)
idata_reg = az.from_cmdstanpy(posterior=fit_reg)

idata_normal.to_netcdf('fit_hierarchical_normal.nc')
idata_reg.to_netcdf('fit_hierarchical_regression.nc')
print('Saved NetCDF files fit_hierarchical_normal.nc and fit_hierarchical_regression.nc')

print('\n=== Summary: Hierarchical Normal (mu, tau, sigma, theta) ===')
print(az.summary(idata_normal, var_names=['mu', 'tau', 'sigma', 'theta'], round_to=2).head(10))

print('\n=== Summary: Hierarchical Regression (mu_alpha, tau_alpha, sigma, alpha, beta) ===')
print(az.summary(idata_reg, var_names=['mu_alpha', 'tau_alpha', 'sigma', 'alpha', 'beta'], round_to=2).head(10))

print('\n=== R-hat diagnostics (Normal model) ===')
rhat_normal = az.rhat(idata_normal)
print(rhat_normal)

print('\n=== R-hat diagnostics (Regression model) ===')
rhat_reg = az.rhat(idata_reg)
print(rhat_reg)


Converting to ArviZ InferenceData...
Saved NetCDF files fit_hierarchical_normal.nc and fit_hierarchical_regression.nc

=== Summary: Hierarchical Normal (mu, tau, sigma, theta) ===
             mean     sd   hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
mu        4504.02  68.66  4381.26  4637.60       2.27     1.26    918.68   
tau          4.00   2.85     0.17     9.12       0.07     0.06   1200.34   
sigma     1938.56  51.10  1844.06  2034.78       1.09     0.80   2186.79   
theta[0]  4503.97  68.66  4373.14  4629.61       2.27     1.26    919.24   
theta[1]  4504.03  68.66  4382.30  4638.37       2.26     1.26    919.47   
theta[2]  4504.04  68.67  4373.05  4629.42       2.27     1.26    918.98   
theta[3]  4504.01  68.67  4366.77  4623.03       2.27     1.26    918.45   

          ess_tail  r_hat  
mu         1699.03    1.0  
tau        1286.07    1.0  
sigma      2225.48    1.0  
theta[0]   1704.37    1.0  
theta[1]   1710.97    1.0  
theta[2]   1699.53    1.0  
theta[3]   1712

In [9]:
# LOO model comparison
print('=== LOO comparison between Hierarchical Normal and Hierarchical Regression ===')

loo_normal = az.loo(idata_normal, pointwise=True)
loo_reg = az.loo(idata_reg, pointwise=True)

print('\nLOO - Hierarchical Normal:')
print(loo_normal)

print('\nLOO - Hierarchical Regression:')
print(loo_reg)

comp = az.compare({'Hierarchical Normal': idata_normal, 'Hierarchical Regression': idata_reg})
comp


=== LOO comparison between Hierarchical Normal and Hierarchical Regression ===

LOO - Hierarchical Normal:
Computed from 8000 posterior samples and 731 observations log-likelihood matrix.

         Estimate       SE
elpd_loo -6571.41    14.74
p_loo        1.52        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.70]   (good)      731  100.0%
   (0.70, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%


LOO - Hierarchical Regression:
Computed from 8000 posterior samples and 731 observations log-likelihood matrix.

         Estimate       SE
elpd_loo -6571.47    14.74
p_loo        1.65        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.70]   (good)      731  100.0%
   (0.70, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%



,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Hierarchical Normal,0,-6571.406821,1.518343,0.000000,1.0,14.738650,0.000000,False,log
Hierarchical Regression,1,-6571.467916,1.648349,0.061095,0.0,14.735971,0.008731,False,log
